//SPDX-License-Identifier: GPL-3.0-or-later

pragma solidity ^0.8.1;// 그냥 원하는 버전

pragma experimental ABIEncoderV2;// 실험용 기능 사용시



In [1]:
%%writefile SimpleChild.sol

//SPDX-License-Identifier: GPL-3.0-or-later
pragma solidity ^0.8.0;

contract Parent {
    //state variables
    address owner; //as of 0.8.0 no need to be address payable owner;
    uint private counter;
    
    //constructor. no need for constructor to be public as of 0.7.0
    constructor() {
        owner=msg.sender;
        counter = 0;
    }
    //functions
    function add() public { counter++; }
    function getCounter() public view returns(uint) { return counter; }
}

contract SimpleChild is Parent {// 상속
    //state variables
    string nickName;
    mapping(address => uint) private balances;// json같은 형식 -> mapping
    //event
    event PrintLog(address, uint);// 함수를 이벤트로 발생시켜 로그를 볼 수 있음
    
    //constructor
    constructor() {}  // no public as in constructor() public {}  
    //functions
    function setNickName(string memory s) public { nickName = s; }
    function getNickName() public view returns(string memory) { return nickName; }
    function deposit() public payable {// 결제시 사용
        balances[msg.sender] += msg.value;// value 값을 추가해줌
        emit PrintLog(msg.sender, msg.value);
    }
    function queryBalance() public view returns (uint) {
        return balances[msg.sender];// 잔고를 읽어옴
    }
    //access non-private members of the parent
    function kill() public {
        if (msg.sender == owner) selfdestruct(payable(owner)); //0.6.x selfdestruct(owner)
    }
}

Writing SimpleChild.sol


 Solidity에서 파일명은 컨트랙명과 일치하지 않아도 되는데, 파일에 하나 이상의 컨트랙트를 포함할 수 있기 때문에 그렇다. 파일명과 컨트랙명은 일치시키는 것을 권고한다.
* import문은 프로그램의 시작 부분에 작성하되, 프로그램 위, pragma 다음 줄에 적어준다.
* 컨트랙은 새로운 줄에 시작한다.
* 컨트랙, event, enum, struct은 낙타 표기법(camel case)을 따라 단어의 첫글자는 대문자로 작성한다.
* 함수, 함수인자, 변수, 함수 변경자(modifier)는 소문자로 시작하고 낙타 표기법으로 작성한다.
* 상수는 모두 대문자로 작성하되, 단어 사이를 밑줄 문자(underscore)로 연결해서 적어준다 (예: DATE_OF_BIRTH).
* 한 파일에 여러 컨트랙을 포함할 경우, 컨트랙 간에는 2줄 띄어쓰기를 해준다.
* 들여쓰기 할 때 탭 문자를 사용하지 말고 공백 문자 4칸을 넣어주자.
* 배열은 ```int[] x;```이라고 적어준다 (```int [] x``` 또는 ```int x[]```가 아니라)
* 문자열은 쌍따옴표로 표시한다. 
* 함수는 새로운 줄에 작성한다.
* 한 줄은 최대 79 문자를 넘지 않도록 하자. 80x25
* 괄호에서는 한 칸 띄어쓰기를 하지 않는다. ```if (x == 1)``` (```if ( x == 1 )```이 아니라)
* 연산자 앞 뒤에는 공백 1칸을 넣는다.
* 코드 블록을 적을 때, 다음과 같이 줄바꿈 없이 중괄호를 연결한다.

## 1.4 예약어

예약어(reserved word)는 키워드(keyword)라고 부르기도 하며 Solidity 언어에서 특별한 목적으로 사용하기 위해 지정한 단어들이다. 이런 명령어는 컨트랙이나 변수를 명명할 때 사용하지 않는다.

다음은 예약어를 알파벳 순으로 나열한 것이다.

abstract, after, alias, apply, auto, byte, case, catch, constant, copyof, default,
define, final, fixed, immutable, implements, import, in, inline, interface,
let, macro, match, mutable, null, of, override, partial, pragma, promise, reference, relocatable, returns, rollback, sealed, sizeof, static, supports, switch, try, type, 
typedef, typeof, using, var, virtual, weak, unchecked

Solidity 언어에는 더 많은 키워드가 있을 수 있으며, 새로운 버전이 출시될 때마다 업데이트 될 수 있다.

# 2. 데이터 타입

Solidity 언어는 공간이 제한된 블록체인에서 실행되는 까닭에 **저장공간을 효율적**으로 사용하는 편이 좋다. 저장공간을 많이 사용할수록 비용이 증가한다. - 변수 설정도 주의

다른 언어에서 지원하는 데이터타입을 큰 차이 없이 사용할 수 있지만, 메모리를 가급적 적게 사용할 수 있도록 설계되어 있어 불필요한 낭비를 줄이고 있다. 메모리는 실로 중요한 자원이고, 블록체인에서 불필요하게 또는 과도하게 ```배열을 검색```하거나 ```반복문```을 사용하는 것을 유의해야 한다.

그리고 소수점은 아직 지원되지 않아 float, double과 같은 자료형이 없다. 또한 암호화폐를 가지고 있어, 계정 주소 타입이 있다는 점이 특별나다.

 자료형    | 설명
----------|----------
bool | 내부적으로 uint8로 표현됨(0 또는 1을 나타냄). 거짓(false)은 0으로 참(true)은 1로 표현된다. 
uint<M> | 양의 정수(unsigned integer), M은 0 ~ 256비트까지, 0 < M <= 256, M은 8의 배수만 사용 가능. 
int<M> | 부호가 있는 정수(signed integer), M은 0 ~ 256비트까지 가능하다 0 < M <= 256, M은 8의 배수만 사용 가능. 
address | 주소. 크기가 20바이트이므로 **uint160** 이다.
uint, int | 숫자가 붙지 않은 경우의 타입으로 uint256, int256를 의미
bytes<M> | 바이너리 타잎, M은 32바이트까지 가능하다 0 < M <= 32. 크기를 지정하지 않는 **```bytes```는 value type이 아니다**.// 고정길이
string | UTF-8 문자열, 동적으로 크기가 정해지므로 **value type이 아니다**. //가변길이

bool isMarried=true;

int256 x=1;

소수 계산 지원 x OpenZeppelin의 SafeMath 라이브러리에 부동 소수점 연산

실행할때 solidity에서 변수형이 어떻게 받아오는가

bytes1 x= 0xFF;
bytes23 place1 = "7 hongji-dong jongro-gu"; //23글자에 맞추어 bytes23으로 선언
bytes8 place2 = "7 hongji"; //8글자에 맞추어 bytes8로 선언.

바이트는 문자열을 사용해도 되는데 알아서 바이트 형태로 바꿔줌

In [ ]:
# mvc
# model view controller 3계층 -> 덤터미널 -> 80줄???

In [3]:
%%writefile IntBool.sol

//SPDX-License-Identifier: GPL-3.0-or-later
pragma solidity ^0.8.0;

contract IntBoolTest {
    bool married = true;
    uint256 xAge = 22;
    uint256 yAge = 25;
    //fixed phi; // = 3.14; // 선언만 가능
    function update() public {
        xAge = yAge;
        yAge = 33;
    }
    function setXAge(int age) public {
        xAge = uint(age);  //type conversion
    }
    function getXAge() public view returns(uint) {
        return xAge;
    }
    function getYAge() public view returns(uint) {
        return yAge;
    }
    function testInt() public view returns(bool) {
        assert(xAge>=20 && yAge>=20);
        return true;
    }
    function isMarried() public view returns(bool) {
        return married;
    }
}

Writing IntBool.sol
